In [ ]:
import matplotlib.pyplot as plt
from utilities import *
import json

In [ ]:
### Get an overview of the histograms calculated.
values = run_sql('value_data', index_col='value_id')

# get static histogram bins
bins = run_sql('get_bins', index_col='id')

display(bins, values)

values.to_csv('../export/value_overview.csv')

In [ ]:
## loop through all vehicles, charging and driving
# get all available 1D-Histograms and save them as JSON objects

for value_id in values.index.values:
    # set a value_id to compute following information...
    label_txt = f"{values.loc[value_id].name_de} / {values.loc[value_id].unit}"
    print(f"Getting histograms for value id {value_id}, {label_txt}")

    for state in ['charging', 'track']:
        # fetch data
        df = run_sql(f'{state}_histograms', params={'value_id':value_id}, cache_result=(value_id!=900))
        
        # Convert counts to numpy arrays for easier manipulation
        df['counts'] = df['counts'].apply(np.array)  

        # sum up the individual "track histograms" per vehicle
        # result = df.groupby('vehicle_id')['counts'].apply(lambda x: np.sum(np.array(list(x)), axis=0)).reset_index()
        # result.set_index('vehicle_id', inplace=True)
        result_series = df.groupby('vehicle_id').apply(lambda x: x.counts.apply(pd.Series).sum().values)
        result = pd.DataFrame(result_series, columns = ['counts'])
        for vehicle_id, data in result.iterrows():
            # look up the binning system
            bins_id = df.bins_id_x.values[0]

            # epsilon-fix so histogram does not exhibit harmonic artifacts
            eps = 0.00000001* (value_id in (900,1302,1303,1288))
            # look up the physical values encoded in the bins
            x_values = np.arange(bins.loc[bins_id].minimum, 
                                 bins.loc[bins_id].maximum + eps,
                                 bins.loc[bins_id].stepsize)
            y_values = data.counts
            assert len(x_values) == len(y_values), "dimension mismatch"
            # plot everything
            if value_id == 4: #and vehicle_id == 635:
                xv, yv = x_values, y_values
                plt.plot(x_values,y_values / y_values.sum(), label=f"{vehicle_id}-{state}")
                plt.xlabel(f'{state} {label_txt}')
                plt.legend()

            data = {'value_id': int(value_id), 'vehicle_id': int(vehicle_id), 'state': state, 'value_description': values.loc[value_id].to_dict(),'data': {'x_values': list(x_values), 'y_values': list(y_values)}}

            # Write dictionary to JSON file
            if False:
                with open(f"../export/json/{state}_{vehicle_id}_{value_id}.json", "w") as json_file:
                    try:
                        json.dump(data, json_file, indent=4)
                    except Exception as e:
                        print(data)
                        raise e

In [ ]:
import matplotlib.pyplot as plt
from utilities import *
import json

In [ ]:
### Get an overview of the histograms calculated.
values = run_sql('value_data', index_col='value_id')

# get static histogram bins
bins = run_sql('get_bins', index_col='id')

display(bins, values)

values.to_csv('../export/value_overview.csv')

In [ ]:
## loop through all vehicles, charging and driving
# get all available 1D-Histograms and save them as JSON objects

for value_id in values.index.values:
    # set a value_id to compute following information...
    label_txt = f"{values.loc[value_id].name_de} / {values.loc[value_id].unit}"
    print(f"Getting histograms for value id {value_id}, {label_txt}")

    for state in ['charging', 'track']:
        # fetch data
        df = run_sql(f'{state}_histograms', params={'value_id':value_id}, cache_result=(value_id!=900))
        
        # Convert counts to numpy arrays for easier manipulation
        df['counts'] = df['counts'].apply(np.array)  

        # sum up the individual "track histograms" per vehicle
        # result = df.groupby('vehicle_id')['counts'].apply(lambda x: np.sum(np.array(list(x)), axis=0)).reset_index()
        # result.set_index('vehicle_id', inplace=True)
        result_series = df.groupby('vehicle_id').apply(lambda x: x.counts.apply(pd.Series).sum().values)
        result = pd.DataFrame(result_series, columns = ['counts'])
        for vehicle_id, data in result.iterrows():
            # look up the binning system
            bins_id = df.bins_id_x.values[0]

            # stupid fix so bins match
            eps = 0.00000001* (value_id in (900,1302,1303,1288))
            # look up the physical values encoded in the bins
            x_values = np.arange(bins.loc[bins_id].minimum, 
                                 bins.loc[bins_id].maximum + eps,
                                 bins.loc[bins_id].stepsize)
            y_values = data.counts
            assert len(x_values) == len(y_values), "dimension mismatch"
            # plot everything
            if value_id == 900 and vehicle_id == 635:
                xv, yv = x_values, y_values
                plt.plot(x_values,y_values / y_values.sum(), label=f"{vehicle_id}-{state}")
                plt.xlabel(f'{state} {label_txt}')
                plt.legend()

            data = {'value_id': int(value_id), 'vehicle_id': int(vehicle_id), 'state': state, 'value_description': values.loc[value_id].to_dict(),'data': {'x_values': list(x_values), 'y_values': list(y_values)}}

            # Write dictionary to JSON file
            if False:
                with open(f"../export/json/{state}_{vehicle_id}_{value_id}.json", "w") as json_file:
                    try:
                        json.dump(data, json_file, indent=4)
                    except Exception as e:
                        print(data)
                        raise e